In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
movies = pd.read_csv("../input/movie_metadata.csv")
movies.info()
movies.head()[:2]
movies.columns
movies.describe()

In [ ]:
print(movies.shape)

In [ ]:
movie_nonobj=movies.dtypes[movies.dtypes!='object'].index
movie_train=movies[movie_nonobj]
movie_train.head()[:4]
movie_train.info()
movie_train.shape
movie_train.describe()

In [ ]:
np.sum(movie_train.isnull())
movie_train=movie_train.fillna(0)
movie_cols=movie_train.columns.tolist()
y=movie_train['imdb_score']
movie_train.drop(['imdb_score'],axis=1,inplace=True)
movie_train.head()[:2]


In [ ]:
mov_corr_mat=movie_train.corr(method='pearson')
plt.figure(figsize=(20,10))
sns.heatmap(mov_corr_mat,vmax=1,square=True,annot=True,cmap='cubehelix')

In [ ]:
from sklearn.preprocessing import StandardScaler
movie_Train=movie_train.values
movie_Train=np.asarray(movie_Train)

# Finding normalised array of X_Train
movie_norm=StandardScaler().fit_transform(movie_Train)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(movie_norm)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,7,1)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')


In [ ]:
from sklearn.decomposition import PCA
sklearn_pca=PCA(n_components=5)
movie_Train=sklearn_pca.fit_transform(movie_norm)
sns.set(style='darkgrid')
m, axis = plt.subplots(figsize=(8, 8))

axis = sns.kdeplot(movie_Train[:,0], movie_Train[:,1], cmap="Greens",
          shade=True, shade_lowest=False)
axis = sns.kdeplot(movie_Train[:,1],movie_Train[:,2], cmap="Reds",
          shade=True, shade_lowest=False)
axis = sns.kdeplot(movie_Train[:,2], movie_Train[:,3], cmap="Blues",
          shade=True, shade_lowest=False)
red = sns.color_palette("Reds")[-2]
blue = sns.color_palette("Blues")[-2]
green = sns.color_palette("Greens")[-2]
axis.text(0.5, 0.5, "2nd and 3rd Projection", size=12, color=blue)
axis.text(-4, 0.0, "1st and 3rd Projection", size=12, color=red)
axis.text(2, 0, "1st and 2nd Projection", size=12, color=green)
plt.xlim(-6,5)
plt.ylim(-2,2)

In [ ]:
number_of_samples = len(y)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.8)
mx_train = movie_Train[random_indices[:num_training_samples]]
my_train=y[random_indices[:num_training_samples]]
mx_test=movie_Train[random_indices[num_training_samples:]]
my_test=y[random_indices[num_training_samples:]]
my_Train=list(my_train)

In [ ]:
# Ridge Regresson
from sklearn import linear_model
model=linear_model.Ridge()
model.fit(mx_train,my_train)
my_predict=model.predict(mx_train)

error=0
for i in range(len(my_Train)):
    error+=(abs(my_Train[i]-my_predict[i])/my_Train[i])
train_error_ridge=error/len(my_Train)*100
print("Train error = "'{}'.format(train_error_ridge)+" percent in Ridge Regression")

y_test=model.predict(mx_test)
y_Predict=list(my_test)

error=0
for i in range(len(my_test)):
    error+=(abs(y_Predict[i]-y_test[i])/y_Predict[i])
test_error_ridge=error/len(y_test)*100
print("Test error = "'{}'.format(test_error_ridge)+" percent in Ridge Regression")


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model.predict(mx_train), "true":my_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title("Residual plot in Ridge Regression")

In [ ]:
#Baysian Regression
reg = linear_model.BayesianRidge()
reg.fit(mx_train,my_train)
y1_reg=reg.predict(mx_train)
y1_reg=list(y1_reg)
y2_reg=reg.predict(mx_test)
y2_reg=list(y2_reg)

error=0
for i in range(len(my_train)):
    error+=(abs(y1_reg[i]-my_Train[i])/my_Train[i])
train_error_bay=error/len(my_Train)*100
print("Train error = "+'{}'.format(train_error_bay)+" percent"+" in Bayesian Regression")

error=0
for i in range(len(my_test)):
    error+=(abs(y2_reg[i]-y_test[i])/y_test[i])
test_error_bay=(error/len(y_test))*100
print("Test error = "+'{}'.format(test_error_bay)+" percent"+" in Bayesian Regression")

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":reg.predict(mx_train), "true":my_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title("Residual plot in Bayesian Regression")

In [ ]:
# KNN Regression
from sklearn import neighbors
n_neighbors=5
knn=neighbors.KNeighborsRegressor(n_neighbors,weights='uniform')
knn.fit(mx_train,my_train)
y1_knn=knn.predict(mx_train)
y1_knn=list(y1_knn)

error=0
for i in range(len(my_train)):
    error+=(abs(y1_knn[i]-my_Train[i])/my_Train[i])
train_error_knn=error/len(my_Train)*100
print("Train error = "+'{}'.format(train_error_knn)+" percent"+" in Knn algorithm")

y2_knn=knn.predict(mx_test)
y2_knn=list(y2_knn)
error=0
for i in range(len(my_test)):
    error+=(abs(y2_knn[i]-y_test[i])/y_test[i])
test_error_knn=error/len(y_test)*100
print("Test error = "'{}'.format(test_error_knn)+" percent"+" in knn algorithm")

In [ ]:
import matplotlib

matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":knn.predict(mx_train), "true":my_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title("Residual plot in Knn")

In [ ]:
#Decission tree regression        
from sklearn import tree
dec = tree.DecisionTreeRegressor(max_depth=1)
dec.fit(mx_train,my_train)
y1_dec=dec.predict(mx_train)
y1_dec=list(y1_dec)
y2_dec=dec.predict(mx_test)
y2_dec=list(y2_dec)

error=0
for i in range(len(my_train)):
    error+=(abs(y1_dec[i]-my_Train[i])/my_Train[i])
train_error_tree=error/len(my_Train)*100
print("Train error = "+'{}'.format(train_error_tree)+" percent"+" in Decision Tree Regressor")

error=0
for i in range(len(my_test)):
    error+=(abs(y1_dec[i]-y_test[i])/y_test[i])
test_error_tree=error/len(y_test)*100
print("Test error = "'{}'.format(test_error_tree)+" percent in Decision Tree Regressor")

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":dec.predict(mx_train), "true":my_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title("Residual plot in Decision Tree")

In [ ]:
#SVM Regressor
from sklearn import svm
svm_reg=svm.SVR()
svm_reg.fit(mx_train,my_train)
y1_svm=svm_reg.predict(mx_train)
y1_svm=list(y1_svm)
y2_svm=svm_reg.predict(mx_test)
y2_svm=list(y2_svm)

error=0
for i in range(len(my_train)):
    error+=(abs(y1_svm[i]-my_Train[i])/my_Train[i])
train_error_svm=error/len(my_Train)*100
print("Train error = "+'{}'.format(train_error_svm)+" percent"+" in SVM Regressor")

error=0
for i in range(len(my_test)):
    error+=(abs(y2_svm[i]-y_test[i])/y_test[i])
test_error_svm=error/len(y_test)*100
print("Test error = "'{}'.format(test_error_svm)+" percent in SVM Regressor")

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":svm_reg.predict(mx_train), "true":my_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")
plt.title("Residual plot in SVM")

In [ ]:
import pandas as pd
train_error=[train_error_ridge,train_error_knn,train_error_bay,train_error_tree,train_error_svm]
test_error=[test_error_ridge,test_error_knn,test_error_bay,test_error_tree,test_error_svm]

col={'Train Error':train_error,'Test Error':test_error}
models=['Ridge Regression','Knn','Bayesian Regression','Decision Tree','SVM']
error_matching=pd.DataFrame(data=col,index=models)
error_matching

In [ ]:
Looks KNN has the best Model